# Table des Matières
1. [Introduction et Sélection des données](#intro)
2. [Exploration et traitement des données](#traitement)
3. [Modélisation et évaluation](#model)
4. [Comunication des résultats](#comm)
5. [Retour d'expérience](#retour) 

  
 

---

## Introduction et sélection des données <a class="anchor" id="intro"></a>

Dans le cadre de notre projet de machine Learning, nous allons traiter un sujet d'apprentissage supervisé à partir de données ouvertes. Tout au long de notre projet, nous utiliserons l'IA générative (ChatGPT) pour nous aider dans notre travail. 

Dans un premier temps, nous commençons par demander à ChatGPT des idées de sujets.

Jeu de données ML généré par l'IA 
- DPE (Analyse énergétique, classification des logements)
- Transports en commun (prédiction affluence, optimisation trajet)
- Pollution (prédiction des pics de pollution, classification zones polluées)
- Reconnaissance d'image
- Traitement du langage naturel
- Donnée de santé
- Donnée financière économique (prédiction de tendance..)

Analyse critique de l'IA
L'IA propose souvent des sujets très classiques, déjà réalisé. Il propose même des projets déjà tout fait sur Kaggle. Il manque d'originalité ! 

### Données initiales <a class="anchor" id="intro"></a>


**Changement de Problématique : Explication**

Initialement de notre projet portait sur la prédiction des passoires thermiques à partir des données DPE : **Quels logements sont susceptibles d’être classés comme passoires thermiques (classe F ou G) ?** 

Cependant, après une analyse approfondie des données, nous avons décidé de changer de jeu de données et de problématique.

La problématique initiale, s'est révélée trop simple pour un sujet de machine learning. Les données DPE sont directement corrélées avec les passoires thermiques, ce qui limite les défis en termes de modélisation et d'analyse.

Nous avons donc opté pour une nouvelle problématique, avec un jeu de données moins binaires. 


**Problématique générée avec L'IA sur les données DPE**

- Peut-on prédire la consommation énergétique d’un logement à partir de ses caractéristiques issues du DPE ?
- Quels logements sont susceptibles d’être classés comme passoires thermiques (classe F ou G) ?
- Peut-on prédire la classe énergétique (A à G) d’un logement à partir des caractéristiques fournies par le DPE ?

**Critique de l'IA sur notre approche précedente**

Votre approche est bien structurée et inclut des variables clés dans des domaines pertinents pour prédire si un logement est une passoire thermique. 

Points positifs :
Les variables sélectionnées correspondent bien à des déterminants majeurs de la performance énergétique.
Vous incluez à la fois des variables quantitatives (ex. : consommation énergétique, déperditions) et qualitatives (ex. : type d'installation de chauffage, zone climatique), ce qui permet une modélisation riche.

Critiques et suggestions :
Les bases de données DPE peuvent contenir des valeurs manquantes ou aberrantes (ex. : surfaces non renseignées, consommations incohérentes).
Vérifiez les distributions des variables pour détecter des anomalies.
Plusieurs variables peuvent être fortement corrélées. Une analyse des corrélations peut être utile.
Les passoires thermiques (F et G) pourraient représenter une minorité des données. Cela peut biaiser l’entraînement du modèle.
Les bâtiments anciens sont souvent surreprésentés parmi les passoires thermiques, ce qui peut conduire le modèle à négliger d'autres facteurs. Une analyse exploratoire approfondie est nécessaire.
Les algorithmes robustes pour ce type de problème incluent :
 Forêts aléatoires (Random Forests): pour leur interprétabilité et leur capacité à gérer des variables mixtes.
 Gradient Boosting (XGBoost, LightGBM) : pour leur performance sur des ensembles déséquilibrés.
 Réseaux de neurones : si les données sont enrichies avec des caractéristiques complexes (géographiques, temporelles).

lien données DPE : https://drive.google.com/file/d/1nUbA6m3SQ9PXpc9i9tD6yADe0fDFYKTc/view?usp=drive_link


### Données définitives <a class="anchor" id="intro"></a>

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
# Scikit-learn
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix

In [ ]:
df_flight = pd.read_csv("data/itineraries_sample.csv")

Afin de répondre à notre problématique, nous utilisons un jeu de données issu de vols aller simple du site Expedia entre le 16 avril 2022 et le 5 octobre 2022. 

Voici l'intégralité des variables présentes dans notre jeu de données : 

- **Identifiants et Dates**  
  - `legId` : Identifiant unique pour chaque vol.  
  - `searchDate` : Date de la recherche effectuée sur Expedia (AAAA-MM-JJ).  
  - `flightDate` : Date du vol (AAAA-MM-JJ).  

- **Informations sur les Aéroports**  
  - `startingAirport` : Code IATA (trois caractères) de l’aéroport de départ.  
  - `destinationAirport` : Code IATA (trois caractères) de l’aéroport d’arrivée.  

- **Tarification et Conditions**  
  - `fareBasisCode` : Code de base tarifaire.  
  - `baseFare` : Tarif de base du billet (en USD).  
  - `totalFare` : Tarif total incluant taxes et frais (en USD).  
  - `isBasicEconomy` : Indique si le billet appartient à la classe économique basique (booléen).  
  - `isRefundable` : Indique si le billet est remboursable (booléen).  

- **Caractéristiques du Vol**  
  - `isNonStop` : Indique si le vol est direct (booléen).  
  - `travelDuration` : Durée totale du trajet (heures et minutes).  
  - `elapsedDays` : Nombre de jours écoulés avant le vol.  
  - `seatsRemaining` : Nombre de sièges restants disponibles.  
  - `totalTravelDistance` : Distance totale parcourue en miles.  

- **Segments du Vol**  
Les données contiennent également des informations détaillées sur chaque segment du trajet :  
  - Heures de départ et d’arrivée (`segmentsDepartureTimeRaw`, `segmentsArrivalTimeRaw`).  
  - Codes aéroportuaires des départs et arrivées (`segmentsDepartureAirportCode`, `segmentsArrivalAirportCode`).  
  - Compagnies aériennes et avions utilisés (`segmentsAirlineName`, `segmentsEquipmentDescription`).  
  - Durée et distance de chaque segment (`segmentsDurationInSeconds`, `segmentsDistance`).  
  - Cabine (`segmentsCabinCode`).  


In [ ]:
df_flight.info()

**Sélection des données** 

Dans un premier temps, nous sélectionnons les variables qui nous semblent pertinantes. 

Voici la liste des variables que nous retenons : 
- `legId`
- `searchDate`
- `flightDate`
- `startingAirport`
- `destinationAirport`
- `fareBasisCode`
- `travelDuration`
- `elapsedDays`
- `isBasicEconomy`
- `isRefundable`
- `isNonStop`
- `totalFare`
- `seatsRemaining`
- `totalTravelDistance`


In [ ]:
var_select = [
'legId',
'searchDate',
'flightDate',
'startingAirport',
'destinationAirport',
'fareBasisCode',
'travelDuration',
'elapsedDays',
'isBasicEconomy',
'isRefundable',
'isNonStop',
'totalFare',
'seatsRemaining',
'totalTravelDistance'
]

## Exploration et traitement des données <a class="anchor" id="traitement"></a>

### Traitement des valeurs manquantes <a class="anchor" id="traitement"></a>

In [ ]:
pourcentage_valeurs_manquantes = df_flight.isnull().mean() * 100
pourcentage_valeurs_manquantes

In [ ]:
# Retirer toutes les lignes contenant des valeurs manquantes
df_cleaned = df_flight.dropna()

# Vérifier que les valeurs manquantes ont été supprimées
print(df_cleaned.isnull().sum())

In [ ]:
df_cleaned = df_cleaned[var_select]
df_cleaned

### Corrélation entre les variables <a class="anchor" id="traitement"></a>

### Répartion des variables <a class="anchor" id="traitement"></a>

Modèle

In [36]:
import pandas as pd
import numpy as np

# scikit-learn
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, LabelEncoder, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

# Modèles
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from xgboost import XGBRegressor

# Métriques
from sklearn.metrics import mean_squared_error, r2_score

# =====================

# Définition de la cible (y) et des features (X)
target = 'totalFare'

features = [
    "legId", "searchDate", "flightDate", "startingAirport", "destinationAirport",
    "fareBasisCode", "travelDuration", "elapsedDays", "isBasicEconomy",
    "isRefundable", "isNonStop", "seatsRemaining", "totalTravelDistance"
]

X = df_cleaned[features].copy()
y = df_cleaned[target].copy()

# Séparation en train et test
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

# =====================
# Colonnes booléennes : on les convertit en int (0/1) si ce n'est pas déjà fait
bool_cols = ["isBasicEconomy", "isRefundable", "isNonStop"]
for col in bool_cols:
    X_train[col] = X_train[col].astype(int, errors="ignore")
    X_test[col] = X_test[col].astype(int, errors="ignore")

numeric_features = ["elapsedDays", "seatsRemaining", "totalTravelDistance"] + bool_cols

categorical_features = [
    "legId",
    "searchDate",
    "flightDate",
    "startingAirport",
    "destinationAirport",
    "fareBasisCode",
    "travelDuration"
]

from sklearn.compose import make_column_selector

numeric_transformer = Pipeline([
    ("imputer", SimpleImputer(strategy="mean")),
    ("scaler", StandardScaler())
])

categorical_transformer = Pipeline([
    ("imputer", SimpleImputer(strategy="constant", fill_value="missing")),
    ("onehot", OneHotEncoder(handle_unknown="ignore"))
])

preprocessor = ColumnTransformer([
    ("num", numeric_transformer, numeric_features),
    ("cat", categorical_transformer, categorical_features)
])

models = {
    "DecisionTree": DecisionTreeRegressor(random_state=42),
    "RandomForest": RandomForestRegressor(random_state=42),
    "LinearRegression": LinearRegression(),
    "XGBoost": XGBRegressor(random_state=42, verbosity=0)  # verbosity=0 pour éviter les logs
}

results = []

for model_name, model in models.items():
    # Pipeline = Preprocessing + Modèle
    pipe = Pipeline([
        ("preprocessing", preprocessor),
        ("regressor", model)
    ])
    
    # Entraînement
    pipe.fit(X_train, y_train)
    
    # Prédiction
    y_pred = pipe.predict(X_test)
    
    # Evaluation
    mse = mean_squared_error(y_test, y_pred)
    rmse = np.sqrt(mse)
    r2 = r2_score(y_test, y_pred)
    
    results.append({
        "model": model_name,
        "rmse": rmse,
        "r2": r2
    })

# Affichage des résultats
results_df = pd.DataFrame(results)
print(results_df)


: 

: 

In [ ]:
# Feature Engineering

# ----------------------------------------------------------
# 1) Conversion des dates en type datetime (si ce n’est pas déjà fait)
# ----------------------------------------------------------
df['searchDate'] = pd.to_datetime(df['searchDate'], errors='coerce')
df['flightDate'] = pd.to_datetime(df['flightDate'], errors='coerce')

# ----------------------------------------------------------
# 2) daysBetweenSearchAndFlight
#    \text{daysBetweenSearchAndFlight} = \text{flightDate} - \text{searchDate} (en jours)
# ----------------------------------------------------------
df['daysBetweenSearchAndFlight'] = (df['flightDate'] - df['searchDate']).dt.days

# ----------------------------------------------------------
# 3) searchDayOfWeek et flightDayOfWeek
#    Extraire le jour de la semaine (0 = Lundi, 6 = Dimanche)
# ----------------------------------------------------------
df['searchDayOfWeek'] = df['searchDate'].dt.dayofweek
df['flightDayOfWeek'] = df['flightDate'].dt.dayofweek

# ----------------------------------------------------------
# 4) searchMonth et flightMonth
#    Extraire le mois (1 à 12)
# ----------------------------------------------------------
df['searchMonth'] = df['searchDate'].dt.month
df['flightMonth'] = df['flightDate'].dt.month

# ----------------------------------------------------------
# 5) flightWeekend (booléen)
#    True si le jour du vol est un samedi ou dimanche (dayofweek in [5,6])
# ----------------------------------------------------------
df['flightWeekend'] = df['flightDayOfWeek'].isin([5, 6])

# ----------------------------------------------------------
# 7) fareType (classification selon le référentiel Wikipédia)
#    https://en.wikipedia.org/wiki/Fare_basis_code
# ----------------------------------------------------------
# On définit des sets de lettres pour chaque classe tarifaire
FIRST = {'F', 'A', 'P'}
BUSINESS = {'C', 'J', 'D', 'I', 'Z'}
PREMIUM_ECONOMY = {'W', 'S', 'R'}
ECONOMY = {'Y', 'B', 'M', 'U', 'H', 'Q', 'K', 'L', 'G', 'V', 'T', 'N', 'X', 'O', 'E'}

def map_fare_type(code):
    """
    Retourne la cabine probable (First, Business, Premium Economy, Economy, Unknown)
    en se basant sur la première lettre du fareBasisCode.
    """
    # Gérer le cas des NaN ou chaînes vides
    if pd.isnull(code) or not code:
        return 'Unknown'
    
    # On récupère la première lettre, en majuscule
    first_char = str(code).strip().upper()[0]
    
    if first_char in FIRST:
        return 'First'
    elif first_char in BUSINESS:
        return 'Business'
    elif first_char in PREMIUM_ECONOMY:
        return 'Premium Economy'
    elif first_char in ECONOMY:
        return 'Economy'
    else:
        return 'Unknown'

df['fareType'] = df['fareBasisCode'].apply(map_fare_type)

# ----------------------------------------------------------
# 10) isOvernightFlight
#     Si elapsedDays > 0, alors True, sinon False
# ----------------------------------------------------------
# elapsedDays est souvent 0 ou 1+ si on traverse la nuit
df['isOvernightFlight'] = df['elapsedDays'] > 0

# ----------------------------------------------------------
# 18) distanceBands
#     "short-haul" (< 500 miles), "medium-haul" (500-1500 miles), "long-haul" (> 1500 miles)
# ----------------------------------------------------------
def define_distance_band(x):
    if pd.isnull(x):
        return 'unknown'   # ou None, selon préférence
    elif x < 500:
        return 'short-haul'
    elif x <= 1500:
        return 'medium-haul'
    else:
        return 'long-haul'

df['distanceBands'] = df['totalTravelDistance'].apply(define_distance_band)

# ----------------------------------------------------------
# Vérification rapide
# ----------------------------------------------------------
print(df[[
    'searchDate', 'flightDate', 'daysBetweenSearchAndFlight',
    'searchDayOfWeek', 'flightDayOfWeek', 'searchMonth', 'flightMonth',
    'flightWeekend', 'fareBasisCode', 'fareType',
    'elapsedDays', 'isOvernightFlight',
    'totalTravelDistance', 'distanceBands'
]].head())